# Module 01: Setup and First Inference

> **Goal**: Get Dynamo running and make your first inference request in under 15 minutes.

---

## Step 1: Verify Your Environment

Let's make sure we have the basics: GPU, Python, and network access.

In [58]:
# Quick environment check
import subprocess
import sys

print("=" * 60)
print("ENVIRONMENT CHECK")
print("=" * 60)

# Python version
print(f"\n✓ Python: {sys.version.split()[0]}")

# GPU check
try:
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=name,memory.total,driver_version', '--format=csv,noheader'],
        capture_output=True, text=True, timeout=5
    )
    if result.returncode == 0:
        for line in result.stdout.strip().split('\n'):
            print(f"✓ GPU: {line}")
    else:
        print("✗ No GPU detected")
except Exception as e:
    print(f"✗ GPU check failed: {e}")

# CUDA check
try:
    result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        cuda_version = [l for l in result.stdout.split('\n') if 'release' in l][0]
        print(f"✓ CUDA: {cuda_version.split('release')[-1].strip().rstrip(',').strip()}")
except:
    print("! CUDA toolkit not found (may still work with runtime)")

print("\n" + "=" * 60)

ENVIRONMENT CHECK

✓ Python: 3.12.3
✓ GPU: NVIDIA GeForce RTX 5090, 32607 MiB, 580.105.08
✓ GPU: NVIDIA GeForce RTX 5090, 32607 MiB, 580.105.08
! CUDA toolkit not found (may still work with runtime)



---

## Step 2: Install Dynamo

Dynamo can be installed from PyPI. We'll use SGLang as our inference backend.

In [59]:
# Install Dynamo with SGLang (as per official quickstart)
# Note: This may take a few minutes

%uv pip install "ai-dynamo[sglang]"

print("\n" + "=" * 60)
print("Verifying installation...")
print("=" * 60)

# Verify Python imports
try:
    import dynamo
    print(f"✓ Dynamo Python package imported successfully")
except ImportError as e:
    print(f"✗ Dynamo import failed: {e}")

try:
    import torch
    print(f"✓ PyTorch version: {torch.__version__}")
    print(f"✓ CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"✓ CUDA device: {torch.cuda.get_device_name(0)}")
except ImportError as e:
    print(f"✗ PyTorch import failed: {e}")

try:
    import sglang
    print(f"✓ SGLang version: {sglang.__version__}")
except ImportError as e:
    print(f"✗ SGLang import failed: {e}")

/root/src/github.com/sara4dev/ai-dynamo-the-hard-way/.venv/bin/python: No module named uv
Note: you may need to restart the kernel to use updated packages.

Verifying installation...
✓ Dynamo Python package imported successfully
✓ PyTorch version: 2.9.1+cu128
✓ CUDA available: True
✓ CUDA device: NVIDIA GeForce RTX 5090
✓ SGLang version: 0.5.6.post2


---

## Step 3: Start Infrastructure (etcd + NATS)

Dynamo needs two services running:
- **etcd**: Service discovery (workers register here)
- **NATS**: Event messaging (KV cache notifications)

We'll use Docker for quick setup:

In [60]:
%%bash
# Start etcd if not running
if ! docker ps | grep -q dynamo-etcd; then
    echo "Starting etcd..."
    docker run -d \
        --name dynamo-etcd \
        --restart unless-stopped \
        -p 2379:2379 \
        -p 2380:2380 \
        quay.io/coreos/etcd:v3.5.17 \
        /usr/local/bin/etcd \
        --name etcd0 \
        --advertise-client-urls http://0.0.0.0:2379 \
        --listen-client-urls http://0.0.0.0:2379 \
        --initial-advertise-peer-urls http://0.0.0.0:2380 \
        --listen-peer-urls http://0.0.0.0:2380 \
        --initial-cluster etcd0=http://0.0.0.0:2380
    echo "✓ etcd started"
else
    echo "✓ etcd already running"
fi

# Start NATS if not running
if ! docker ps | grep -q dynamo-nats; then
    echo "Starting NATS..."
    docker run -d \
        --name dynamo-nats \
        --restart unless-stopped \
        -p 4222:4222 \
        -p 8222:8222 \
        nats:latest \
        -js -m 8222
    echo "✓ NATS started"
else
    echo "✓ NATS already running"
fi

# Wait for services to be ready
sleep 2
echo ""
echo "Infrastructure status:"
docker ps --format "table {{.Names}}\t{{.Status}}\t{{.Ports}}" | grep -E "(NAMES|dynamo)"

✓ etcd already running
✓ NATS already running

Infrastructure status:
NAMES         STATUS          PORTS
dynamo-nats   Up 41 minutes   0.0.0.0:4222->4222/tcp, [::]:4222->4222/tcp, 0.0.0.0:8222->8222/tcp, [::]:8222->8222/tcp, 6222/tcp
dynamo-etcd   Up 41 minutes   0.0.0.0:2379-2380->2379-2380/tcp, [::]:2379-2380->2379-2380/tcp


In [61]:
# Verify services are responding
import urllib.request
import json

print("Checking service health...\n")

# Check etcd
try:
    with urllib.request.urlopen('http://localhost:2379/health', timeout=5) as response:
        data = json.loads(response.read())
        print(f"✓ etcd: {data}")
except Exception as e:
    print(f"✗ etcd not responding: {e}")

# Check NATS
try:
    with urllib.request.urlopen('http://localhost:8222/healthz', timeout=5) as response:
        status = response.read().decode()
        print(f"✓ NATS: {status}")
except Exception as e:
    print(f"✗ NATS not responding: {e}")

Checking service health...

✓ etcd: {'health': 'true', 'reason': ''}
✓ NATS: {"status":"ok"}


---

## Step 4: Download a Model

We'll use a small model for quick testing: **Qwen3-0.6B**

This is small enough to load quickly but demonstrates all Dynamo functionality.

In [62]:
# Model configuration
MODEL_NAME = "Qwen/Qwen3-0.6B"

print(f"Model: {MODEL_NAME}")
print("\nThis model will be downloaded on first use.")
print("For faster testing, we'll let the backend handle the download.")

Model: Qwen/Qwen3-0.6B

This model will be downloaded on first use.
For faster testing, we'll let the backend handle the download.


---

## Step 5: Start Dynamo Frontend and Worker

Dynamo uses a disaggregated architecture with separate frontend and worker processes:
- **Frontend**: Handles HTTP requests and routes to workers
- **Worker**: Runs the model inference (vLLM backend)

The cell below will start both processes in the background:

In [71]:
%%bash
# Kill any existing Dynamo processes and free port 8000
echo "Killing any existing Dynamo processes..."
pkill -f 'python -m dynamo' 2>/dev/null || true
fuser -k 8000/tcp 2>/dev/null || true
sleep 30

# Start the frontend
echo "Starting Dynamo frontend..."
python -m dynamo.frontend > /tmp/dynamo_frontend.log 2>&1 &

# Start the SGLang worker
echo "Starting Dynamo worker..."
# attention backend flashinfer works with RTX 5090
python -m dynamo.sglang --model-path Qwen/Qwen3-0.6B --attention-backend flashinfer > /tmp/dynamo_worker.log 2>&1 &

echo "Started Dynamo frontend and SGLang worker in background"
echo "Wait ~30s for the model to load before continuing"
echo "Check logs: /tmp/dynamo_frontend.log and /tmp/dynamo_worker.log"

Killing any existing Dynamo processes...
Starting Dynamo frontend...
Starting Dynamo worker...
Started Dynamo frontend and SGLang worker in background
Wait ~30s for the model to load before continuing
Check logs: /tmp/dynamo_frontend.log and /tmp/dynamo_worker.log


In [75]:
import urllib.request
import json
import time

print("Waiting for Dynamo to start...")

MAX_WAIT = 300  # Maximum wait time in seconds
INTERVAL = 5    # Check interval in seconds
elapsed = 0
frontend_ready = False
backend_ready = False

while elapsed < MAX_WAIT:
    try:
        with urllib.request.urlopen('http://localhost:8000/health', timeout=5) as response:
            health = json.loads(response.read())
            
            # Check if frontend is healthy
            if health.get('status') == 'healthy' and not frontend_ready:
                print(f"[{elapsed}s] ✓ Frontend is READY (status: healthy)")
                frontend_ready = True
            
            # Check if backend endpoint is registered
            endpoints = health.get('endpoints', [])
            has_backend = any('backend' in ep for ep in endpoints)
            
            if has_backend:
                print(f"[{elapsed}s] ✓ Backend is READY (endpoints registered)")
                print()
                print("=== Final Health Status ===")
                print(json.dumps(health, indent=2))
                backend_ready = True
                break
            elif frontend_ready:
                print(f"[{elapsed}s] Waiting for backend endpoints... (frontend healthy, no backend yet)")
                
    except Exception as e:
        print(f"[{elapsed}s] Waiting for frontend to respond...")
    
    time.sleep(INTERVAL)
    elapsed += INTERVAL

if not backend_ready:
    print()
    print(f"=== Timeout after {MAX_WAIT}s ===")
    print(f"Frontend ready: {frontend_ready}")
    print(f"Backend ready: {backend_ready}")
    print()
    print("=== Frontend Log (last 20 lines) ===")
    try:
        with open('/tmp/dynamo_frontend.log', 'r') as f:
            lines = f.readlines()
            print(''.join(lines[-20:]))
    except:
        print("No frontend log yet")
    print()
    print("=== Worker Log (last 20 lines) ===")
    try:
        with open('/tmp/dynamo_worker.log', 'r') as f:
            lines = f.readlines()
            print(''.join(lines[-20:]))
    except:
        print("No worker log yet")
else:
    print()
    print("=== Dynamo is fully operational! ===")

Waiting for Dynamo to start...
[0s] ✓ Frontend is READY (status: healthy)
[0s] ✓ Backend is READY (endpoints registered)

=== Final Health Status ===
{
  "status": "healthy",
  "endpoints": [
    "dyn://dynamo.backend.generate"
  ],
  "instances": [
    {
      "component": "backend",
      "endpoint": "generate",
      "namespace": "dynamo",
      "instance_id": 112480155747070545,
      "transport": {
        "tcp": "192.168.1.180:44159/18f9c1b0f95b251/generate"
      }
    }
  ]
}

=== Dynamo is fully operational! ===


---

## Step 6: Your First Inference Request! 🎉

Now let's send a request using the OpenAI-compatible API:

In [76]:
import urllib.request
import json
import time

# Dynamo endpoint
DYNAMO_URL = "http://localhost:8000/v1/chat/completions"

# Request payload (OpenAI-compatible format)
payload = {
    "model": "Qwen/Qwen3-0.6B",
    "messages": [
        {"role": "user", "content": "What is the capital of France? Answer in one sentence."}
    ],
    "max_tokens": 50,
    "temperature": 0.7
}

print("Sending request to Dynamo...")
print(f"Prompt: {payload['messages'][0]['content']}")
print("-" * 50)

start_time = time.time()

try:
    req = urllib.request.Request(
        DYNAMO_URL,
        data=json.dumps(payload).encode(),
        headers={'Content-Type': 'application/json'}
    )
    with urllib.request.urlopen(req, timeout=60) as response:
        result = json.loads(response.read())
        
    elapsed = time.time() - start_time
    
    # Extract response
    answer = result['choices'][0]['message']['content']
    usage = result.get('usage', {})
    
    print(f"\n✓ Response received in {elapsed:.2f}s")
    print(f"\nAnswer: {answer}")
    print(f"\nTokens: {usage.get('prompt_tokens', '?')} prompt + {usage.get('completion_tokens', '?')} completion")
    
except urllib.error.URLError as e:
    print(f"\n✗ Connection failed: {e}")
    print("\nMake sure the Dynamo worker is running (Step 5)")
except Exception as e:
    print(f"\n✗ Error: {e}")

Sending request to Dynamo...
Prompt: What is the capital of France? Answer in one sentence.
--------------------------------------------------

✓ Response received in 3.67s

Answer: <think>
Okay, the user is asking for the capital of France in one sentence. I need to make sure I recall the correct answer. France's capital is Paris. Let me double-check that. Yes, Paris is the capital. I should state

Tokens: 20 prompt + 50 completion


---

## Step 7: Verify System State

Let's check the full system status:

In [77]:
import subprocess
import urllib.request
import json
import base64

print("=" * 60)
print("SYSTEM STATUS")
print("=" * 60)

# GPU utilization
print("\n📊 GPU Status:")
!nvidia-smi --query-gpu=name,memory.used,memory.total,utilization.gpu --format=csv

# Docker containers
print("\n🐳 Infrastructure Containers:")
!docker ps --filter name=dynamo

# Health endpoints
print("\n🏥 Health Checks:")

services = [
    ("Dynamo Frontend", "http://localhost:8000/health"),
    ("etcd", "http://localhost:2379/health"),
    ("NATS", "http://localhost:8222/healthz"),
]

for name, url in services:
    try:
        with urllib.request.urlopen(url, timeout=2) as resp:
            print(f"  ✓ {name}: OK")
    except:
        print(f"  ✗ {name}: Not responding")

# Check registered workers in etcd
# Dynamo registers workers under the "v1/" prefix with two key types:
#   - v1/instances/... : Worker endpoint registration (transport info)
#   - v1/mdc/...       : Model Card with model configuration details
print("\n👷 Registered Workers (from etcd):")
try:
    # Query etcd for all keys under v1/ prefix
    req = urllib.request.Request(
        "http://localhost:2379/v3/kv/range",
        data=json.dumps({
            "key": base64.b64encode(b"v1/").decode(),
            "range_end": base64.b64encode(b"v10").decode()  # range_end for prefix scan
        }).encode(),
        headers={'Content-Type': 'application/json'}
    )
    with urllib.request.urlopen(req, timeout=5) as resp:
        data = json.loads(resp.read())
        if 'kvs' in data and data['kvs']:
            for kv in data['kvs']:
                key = base64.b64decode(kv['key']).decode()
                value = json.loads(base64.b64decode(kv['value']).decode())
                print(f"  ✓ {key}")
                print(json.dumps(value, indent=4))
                print()
        else:
            print("  ⚠ No workers registered yet (wait for model to load)")
except Exception as e:
    print(f"  ✗ Could not query etcd: {e}")

# Verify models endpoint (confirms worker is ready to serve)
print("\n🤖 Available Models:")
try:
    req = urllib.request.Request(
        "http://localhost:8000/v1/models",
        headers={'Content-Type': 'application/json'}
    )
    with urllib.request.urlopen(req, timeout=5) as resp:
        models = json.loads(resp.read())
        if models.get('data'):
            for model in models['data']:
                print(f"  ✓ {model.get('id', 'unknown')}")
        else:
            print("  ⚠ No models available yet")
except Exception as e:
    print(f"  ✗ Could not query models: {e}")

SYSTEM STATUS

📊 GPU Status:
name, memory.used [MiB], memory.total [MiB], utilization.gpu [%]
NVIDIA GeForce RTX 5090, 29808 MiB, 32607 MiB, 0 %
NVIDIA GeForce RTX 5090, 4 MiB, 32607 MiB, 0 %

🐳 Infrastructure Containers:
CONTAINER ID   IMAGE                         COMMAND                  CREATED          STATUS          PORTS                                                                                                NAMES
2a2da28afc9d   nats:latest                   "/nats-server -js -m…"   51 minutes ago   Up 51 minutes   0.0.0.0:4222->4222/tcp, [::]:4222->4222/tcp, 0.0.0.0:8222->8222/tcp, [::]:8222->8222/tcp, 6222/tcp   dynamo-nats
c150bb746ae1   quay.io/coreos/etcd:v3.5.17   "/usr/local/bin/etcd…"   51 minutes ago   Up 51 minutes   0.0.0.0:2379-2380->2379-2380/tcp, [::]:2379-2380->2379-2380/tcp                                      dynamo-etcd

🏥 Health Checks:
  ✓ Dynamo Frontend: OK
  ✓ etcd: OK
  ✓ NATS: OK

👷 Registered Workers (from etcd):
  ✓ v1/instances/dynamo/backend/g

### Understanding the Communication Flow

Looking at the etcd output above, you can see how Dynamo components discover and communicate with each other:

```
Client ──HTTP──▶ Frontend (:8000) ──TCP/gRPC──▶ Worker (registered in etcd)
                     │                              │
                     └──────── etcd ◀───────────────┘
                              (service discovery)
```

**Key entries in etcd:**

| Key Pattern | Purpose | Example Data |
|-------------|---------|--------------|
| `v1/instances/...` | Worker endpoint registration | `transport.tcp`: worker's address for direct communication |
| `v1/mdc/...` | Model Card | Model configuration, context length, runtime settings |

**How the Frontend finds Workers:**

1. Worker starts → registers itself in etcd with its TCP address
2. Frontend queries etcd for `v1/instances/*` to discover available workers
3. Frontend connects directly to worker via TCP (the address in `transport.tcp`)
4. Inference requests flow over this direct TCP connection (not through etcd or NATS)

**Role of each component:**

| Component | Protocol | Purpose |
|-----------|----------|---------|
| **etcd** | HTTP :2379 | "Who exists?" - Service discovery, worker registration |
| **NATS** | TCP :4222 | "What's happening?" - KV cache events, metrics (async) |
| **Frontend→Worker** | TCP/gRPC | Actual inference requests (low-latency direct connection) |

> **Key insight**: NATS is used for event streaming (KV cache notifications), not for request/response. This keeps the inference hot path fast by avoiding message queue latency.

---

## 🎉 Congratulations!

You now have a working Dynamo setup:

| Component | Status | Port |
|-----------|--------|------|
| etcd (Service Discovery) | Running | 2379 |
| NATS (Messaging) | Running | 4222 |
| Dynamo Worker (SGLang) | Running | 8000 |

### What's Next?

- **Module 02**: Deep dive into the Frontend (HTTP handling, routing)
- **Module 03**: Compare different backends (vLLM, SGLang, TensorRT-LLM)
- **Module 04**: Understanding etcd service discovery in detail

---

## Cleanup (Optional)

Run this cell to stop the Dynamo processes when you're done:

In [67]:
%%bash
# Stop Dynamo processes
pkill -f 'python -m dynamo' || true
echo "Dynamo processes stopped"

# Uncomment below to also stop infrastructure containers
# docker stop dynamo-etcd dynamo-nats
# docker rm dynamo-etcd dynamo-nats

Dynamo processes stopped
